## Notebook Magic

In [2]:
%matplotlib inline
%load_ext autoreload

## Imports

In [3]:
import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
import boto3
import time

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_curve

from sklearn.linear_model import LogisticRegression, Lasso

from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import randint

import itertools

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.width = 0

In [4]:
df_train = pd.read_parquet("s3://iefp-unemployment/copy_test_train/train_T180509.parquet")

In [5]:
df_test = pd.read_parquet("s3://iefp-unemployment/copy_test_train/test_T180509.parquet")

In [6]:
s3 = boto3.client('s3')
response = s3.get_object(Bucket='iefp-unemployment', Key='copy_test_train/random_forest_T180509.pkl')
model = pickle.loads(response["Body"].read())

In [7]:
def generate_combinations(n,k):
    result = list()
    for i in range(1, k + 1):
        for bits in itertools.combinations(range(n), i):
                s = [0] * n
                for bit in bits:
                    s[bit] = 1
                result.append(s)

    return pd.DataFrame(result)

In [8]:
def get_user_recommendations(journey_series, model, max_interventions=3, number_results=10, index=1):
    # Get interventions to permute over
    interventions = [col for col in journey_series.axes[0] if col[0:2] == "i_"]
    journey_series.drop(labels=interventions + ["ttj", "ttj_sub_12"], inplace=True)
    
    # Generate permutation dataframe
    permutations = generate_combinations(len(interventions), max_interventions)
    
    # Create user dataframe to match permutation size and join with permutation df
    predict_df = pd.DataFrame(np.tile(journey_series.values,
                              len(permutations.index)).reshape(-1,len(journey_series.index)),
                              columns=journey_series.index).join(permutations)
    
    # Predict for all permutations
    probabilities = pd.DataFrame(model.predict_proba(predict_df))
    probabilities.columns = ["unsuccessful", "successful"]

    #Create final top_n dataframe
    results_df = predict_df.join(probabilities)
    
    top_n = results_df.sort_values(by=['successful'], ascending=False).iloc[0:number_results,:]
    top_n["journey_id"] = index

    return top_n

In [ ]:
def gen_top_recommendations_table(df, model, permutations, number_results):
    # Prepare empty dataframe
    recommendations_df = pd.DataFrame()
    
    # Iterate through each user and append the top 5 recommendations to an empty dataframe
    for index, journey in df.iterrows():
        user_recommendations = get_user_recommendations(journey, index, model, permutations, number_results)
        recommendations_df = pd.concat([recommendations_df,user_recommendations])

    return recommendations_df

In [ ]:
# Test intervention count

small_test = df_test.head(50)

start = time.time()
top_5_10 = gen_top_recommendations_table(small_test, model, new_perms, 20)
print(time.time() - start)

In [ ]:
top_5_10

In [ ]:
matrix = small_test.join(top_5_10.groupby("journey_id").sum().add_prefix('sum_'))

In [ ]:
matrix

In [ ]:
full_set = pd.read_parquet("s3://iefp-unemployment/modelling/modelling.parquet")

In [ ]:
def get_sub_group(df, observation):
    
    journey = pd.DataFrame(observation).T
    
    journey["young"] = (journey["d_age"] >= 0.0) & (journey["d_age"] <= 0.2)
    journey["adult"] = (journey["d_age"] >= 0.21) & (journey["d_age"] <= 0.45)
    journey["middle"] = (journey["d_age"] >= 0.46) & (journey["d_age"] <= 0.7)
    journey["old"] = (journey["d_age"] >= 0.71)
    
    df["young"] = (df["d_age"] >= 0.0) & (df["d_age"] <= 0.3)
    df["adult"] = (df["d_age"] >= 0.31) & (df["d_age"] <= 0.45)
    df["middle"] = (df["d_age"] >= 0.46) & (df["d_age"] <= 0.7)
    df["old"] = (df["d_age"] >= 0.71)
    
    dems = ["d_gender_M", "d_disabled", "d_subsidy", "d_rsi_True",
           "young", "adult", "middle", "old", "d_nationality_other",
           "d_school_qualification_2.0", "d_school_qualification_3.0",
            "d_school_qualification_4.0", "d_school_qualification_5.0",
            "d_school_qualification_6.0", "d_school_qualification_nan"]
    
    sub_group = df.merge(journey, left_on=dems, right_on=dems)

    return sub_group

In [ ]:
filtered = get_sub_group(full_set, full_set.iloc[10])

In [ ]:
filtered